# Day 1. 배열과 알고리즘으로 문제 해결하기 – 심화 미션

이 노트북은 **in-place 배열 조작**과 **two-pointer 전략**을 연습하기 위한 심화 과제용입니다.

## 🎯 학습 목표
- 추가 메모리를 거의 사용하지 않는 **in-place 알고리즘** 개념을 이해한다.
- **two-pointer 패턴(write/read 포인터 등)** 을 실제 문제에 적용해본다.
- 간단한 최적화를 통해 전체 연산 횟수를 줄이는 아이디어를 연습한다.

---
## 오늘의 심화 문제: LeetCode #283 – Move Zeroes

정수 배열 `nums`가 주어질 때,
배열 안의 모든 `0`을 **배열의 뒤쪽으로 이동**시키되,
`0이 아닌 원소들의 상대적 순서는 유지`하도록 배열을 재배치하세요.

이 작업은 **in-place**, 즉 **새로운 배열을 생성하지 않고** 수행해야 합니다.

### 예시
1. 입력: `nums = [0, 1, 0, 3, 12]` → 출력: `[1, 3, 12, 0, 0]`
2. 입력: `nums = [0]` → 출력: `[0]`

### 제한 조건
- `1 <= nums.length <= 10^4`
- `-2^31 <= nums[i] <= 2^31 - 1`

### Follow-up
- 전체 연산(스왑/대입)의 횟수를 어떻게 하면 최소화할 수 있을까요?


## 🛠️ Prerequisites
본 미션에서는 다음과 같은 Python 표준 라이브러리를 사용합니다.
* **`from typing import List`**: `nums` 배열의 타입을 명시하기 위해 사용합니다.
* 별도의 외장 라이브러리 설치 없이 기본 Python 환경에서 실행 가능합니다.

---
## Step 1. Two-pointer로 `moveZeroes` 구현하기

### 아이디어 힌트
- 하나의 포인터는 **현재 읽고 있는 위치(read)** 를 가리킵니다.
- 다른 포인터는 **0이 아닌 값을 쓸 위치(write)** 를 가리킵니다.
- `nums`를 앞에서부터 순회하면서, 0이 아닌 값을 발견하면 `write` 위치에 옮기고 `write`를 한 칸 증가시킵니다.
- 마지막에 `write` 뒤에 남은 위치들을 모두 0으로 채웁니다.

아래 템플릿을 활용해 코드를 완성해 보세요.

In [7]:
from typing import List

class Solution:
  def moveZeroes(self, nums: List[int]) -> None:
      
      write = 0 #쓰기 위치 0 시-작

      for read in range(len(nums)): #읽으러 nums 의 인덱스 길이만큼 순회할 것이다
          if nums[read] != 0: #읽은 위치의 값이 0이 아니면!
              nums[write], nums[read] = nums[read], nums[write] #쓰기위치의 값과 읽기위치의 값을 바꿔라
              write += 1  #그리고나서 읽기는 한 칸 가,, 0이면 어떻게 됨, 그냥가는가?
              #응, 지나감. 조건문 때문에 0이면 아무런 일도 하지 않고, 다음순서로 넘어간다, 읽기 1증가
              #이때, 쓰기는 가만히 있는데, 이 자리가, 나중에 0이 아닌 수를 받아줄 자리가 된다
              #그럼 앞에 모인 수들의 순서는?
              #앞에서부터 차례대로, 먼저 발견한 순서대로 갖다놓기 때문에 먼저 발견된 놈이 무조건 더 앞에 자리한다.
              # 애초 뒤죽박죽이면 이렇게 해도 뒤죽박죽아닌가.
              # 맞아.이건 전체리스트를 크기순으로 정렬-sorting 하는게 아니야
              # 이 알고리즘은 0만 골라서 뒤로 보내고, 나머지 숫자들은 원래순서 그대로 유지해
              # 원본 배열에서 0이 아닌 애들끼리 누가 더 앞에 있었냐 하는 "상대적<순서>"를 따지는 거
      """nums 리스트 안의 모든 0을 뒤로 보내고,
      0이 아닌 원소들의 상대적 순서를 유지하도록 in-place로 수정합니다.

      예를들면, 게시글 목록에서 '광고'나 '삭제된글'은 맨 뒤로 보내주세요, 하지만 남은 게시글들은
      원래 올린 시간순서(원본순서) 그대로 보여줘야 합니다, 같은 경우 투 포인터 전략을 써야한다.

      딱 한 번 순회 하지만! 리스트의 길이에 따라 시간도 늘어나니까! 시간복잡도는 N이다.
      데이터 개수에 비례해서 작업량이 늘어나는 선형 시간(linear time)구조인 것.
      
      브루트포스의 이중 반복문을 한다면, 0을 하나 찾을 때마다 나머지 숫자들을 앞으로 당기고, 
      다시 시작해야 하므로 이것도 N*N 이었겠다.

      그리고, 한 번의 순회동안 본 것을 기록해두지 않았고, 기존리스트의 자리만 맞바꾸었다.
      공간복잡도는 리스트의 길이에 상관없는 부분이다! 다만 "이 알고리즘을 풀기위해 추가로 빌려온
      빈 메모리가 얼마나 큰가?" 이다.
      
      리스트가 10개든 100개든, read 와 write 두개의 변수만 추가로 사용했잖아.
      리스트가 아무리 커져도 새로 만드는 변수의 갯수는 늘어나지 않는다.
      ---데이터 양이 아무리 많아져도 내가 쓰는 추가메모리는 항상 일정하다 는 뜻에서 O(1) 인것.

      정리를 해보면, 
      추가메모리를 안쓰는 알고리즘이지만 O(1), 모든 숫자를 한 번씩은 훑어야 하므로,
      시간은 데이터 길이에 비례해서 든다 O(N).

      시간복잡도 O(N), 공간복잡도 O(1)


      시간복잡도 & 아이디어:
          TODO: 사용한 two-pointer 전략과 시간복잡도(O(N))를 설명해 보세요.
      """
      # TODO: two-pointer(write/read) 방식으로 구현해 보세요.
      #   - 추가 리스트를 만들지 말고(nums를 그대로 수정) in-place로 해결해야 합니다.
      #   - 필요하다면 포인터 변수(write, read)를 두고, 반복문을 한 번만 도는 구현을 시도해 보세요.



### ✅ 테스트 코드
아래 셀을 실행해 다양한 케이스에서 함수가 잘 동작하는지 확인해 보세요.
테스트를 **스스로 1~2개 더 추가**하면 더 좋습니다!

In [9]:
def test_move_zeroes(nums):
    print(f"입력  before: {nums}")
    Solution().moveZeroes(nums)
    print(f"출력  after : {nums}\n")

test_cases = [
    [0, 1, 0, 3, 12],
    [0],
    [1, 2, 3],
    [0, 0, 0, 1],
]

for case in test_cases:
    test_move_zeroes(case)



test_cases2 = [
    [5, 1, 0, 3, 12, 2, 0, 0, 0],
    [0, 100, 20, 0, 0, 2764],
    [1, 2, 3, 0, 0, 3, 2, 1],
    [0, 0, 0, 1],
]

for case2 in test_cases2:
    test_move_zeroes(case2)


입력  before: [0, 1, 0, 3, 12]
출력  after : [1, 3, 12, 0, 0]

입력  before: [0]
출력  after : [0]

입력  before: [1, 2, 3]
출력  after : [1, 2, 3]

입력  before: [0, 0, 0, 1]
출력  after : [1, 0, 0, 0]

입력  before: [5, 1, 0, 3, 12, 2, 0, 0, 0]
출력  after : [5, 1, 3, 12, 2, 0, 0, 0, 0]

입력  before: [0, 100, 20, 0, 0, 2764]
출력  after : [100, 20, 2764, 0, 0, 0]

입력  before: [1, 2, 3, 0, 0, 3, 2, 1]
출력  after : [1, 2, 3, 3, 2, 1, 0, 0]

입력  before: [0, 0, 0, 1]
출력  after : [1, 0, 0, 0]



---
## Step 2. 연산 횟수와 최적화 아이디어

아래에 본인이 구현한 알고리즘의 **시간복잡도와 연산 패턴**을 정리해 보세요.

- 우리는 리스트를 몇 번 순회했나요?
- 불필요한 스왑/대입 연산을 줄일 수 있는 방법은 무엇인가요?
- 두 포인터를 활용함으로써 **전체 복잡도를 어떻게 유지(O(N))** 했는지 설명해 보세요.

---

### ✏️ 정리 (학생 작성)

- 내가 사용한 포인터의 역할 - 전체 시간복잡도와 그 이유:
  -  write = 0 #쓰기 위치 0 시-작

      for read in range(len(nums)): #읽으러 nums 의 인덱스 길이만큼 순회할 것이다
          if nums[read] != 0: #읽은 위치의 값이 0이 아니면!
              nums[write], nums[read] = nums[read], nums[write] #쓰기위치의 값과 읽기위치의 값을 바꿔라
              write += 1  #그리고나서 읽기는 한 칸 가,, 0이면 어떻게 됨, 그냥가는가?
              #응, 지나감. 조건문 때문에 0이면 아무런 일도 하지 않고, 다음순서로 넘어간다, 읽기 1증가
              #이때, 쓰기는 가만히 있는데, 이 자리가, 나중에 0이 아닌 수를 받아줄 자리가 된다
              #그럼 앞에 모인 수들의 순서는?
              #앞에서부터 차례대로, 먼저 발견한 순서대로 갖다놓기 때문에 먼저 발견된 놈이 무조건 더 앞에 자리한다.
              # 애초 뒤죽박죽이면 이렇게 해도 뒤죽박죽아닌가.
              # 맞아.이건 전체리스트를 크기순으로 정렬-sorting 하는게 아니야
              # 이 알고리즘은 0만 골라서 뒤로 보내고, 나머지 숫자들은 원래순서 그대로 유지해
              # 원본 배열에서 0이 아닌 애들끼리 누가 더 앞에 있었냐 하는 "상대적<순서>"를 따지는 거
      """nums 리스트 안의 모든 0을 뒤로 보내고,
      0이 아닌 원소들의 상대적 순서를 유지하도록 in-place로 수정합니다.

      예를들면, 게시글 목록에서 '광고'나 '삭제된글'은 맨 뒤로 보내주세요, 하지만 남은 게시글들은
      원래 올린 시간순서(원본순서) 그대로 보여줘야 합니다, 같은 경우 투 포인터 전략을 써야한다.

      딱 한 번 순회 하지만! 리스트의 길이에 따라 시간도 늘어나니까! 시간복잡도는 N이다.
      데이터 개수에 비례해서 작업량이 늘어나는 선형 시간(linear time)구조인 것.
      
      브루트포스의 이중 반복문을 한다면, 0을 하나 찾을 때마다 나머지 숫자들을 앞으로 당기고, 
      다시 시작해야 하므로 이것도 N*N 이었겠다.

      그리고, 한 번의 순회동안 본 것을 기록해두지 않았고, 기존리스트의 자리만 맞바꾸었다.
      공간복잡도는 리스트의 길이에 상관없는 부분이다! 다만 "이 알고리즘을 풀기위해 추가로 빌려온
      빈 메모리가 얼마나 큰가?" 이다.
      
      리스트가 10개든 100개든, read 와 write 두개의 변수만 추가로 사용했잖아.
      리스트가 아무리 커져도 새로 만드는 변수의 갯수는 늘어나지 않는다.
      ---데이터 양이 아무리 많아져도 내가 쓰는 추가메모리는 항상 일정하다 는 뜻에서 O(1) 인것.

      정리를 해보면, 
      추가메모리를 안쓰는 알고리즘이지만 O(1), 모든 숫자를 한 번씩은 훑어야 하므로,
      시간은 데이터 길이에 비례해서 든다 O(N).

      시간복잡도 O(N), 공간복잡도 O(1)

     

- 연산 횟수를 더 줄일 수 있는 아이디어가 있다면:
  -해시맵 방식으로 이걸 풀면 어떨까,

  해시맵 방식은 '값'을 가지고 '인덱스'를 찾거나 '개수'를 셀 때 쓴댄다. 근데 만약 억지로 해시맵으로 0을 뒤로 보낸다면,

  일단, 리스트 1번 순회하면서 0이 아닌 숫자의 원래 순서를 기록하고

  0이 몇개인지 세고,

  해시맵에 기록한 숫자들을 순서대로 채워넣고,

  세어놓은 0의 갯수만큼 0을 채워넣는다.

  이렇게 하면, 새로 리스트를 만드는 것과 다름이 없네.

시간복잡도는 리스트만큼이니 O(N)이고, 공간복잡도 또한 리스트만큼이니 O(N)이다.
굳이 안써도 될 메모리를 써야 하겠다. 같은 크기만큼.
리스트가 크면 클 수록 쓸데없이 써야할 메모리가 커지는.

결론, 연산 횟수를 줄이려면, 무얼 해야하는지 알아야, 그에 딱 맞는 효율적인 알고리즘을 써야 하겠다. 모르면 물어보자.



---
## 제출 전 체크리스트 (심화)
- [ ] `moveZeroes`가 **in-place**로 동작한다. (새 리스트를 만들지 않았다)
- [ ] 두 포인터 아이디어를 코드에 반영했다.
- [ ] 최소 3개 이상의 테스트 케이스를 실행해 보았다.
- [ ] 시간복잡도와 알고리즘 아이디어를 말/글로 설명할 수 있다.

수고 많았어요! 👏 배열 알고리즘과 two-pointer 패턴은 이후에도 계속 반복해서 등장합니다.


###**콘텐츠 라이선스**
<font color='red'><b>**(주)업스테이지가 제공하는 모든 교육 콘텐츠의 지식재산권은
운영 주체인 (주)업스테이지 또는 해당 저작물의 적법한 관리자에게 귀속되어 있습니다.**</b></font>

콘텐츠 일부 또는 전부를 **복사, 복제, 판매, 재판매 공개, 공유** 등을 할 수 없습니다. 유출될 경우 지식재산권 침해에 대한 책임을 부담할 수 있습니다.

유출에 해당하여 금지되는 행위의 예시는 다음과 같습니다.
* 콘텐츠를 재가공하여 온/오프라인으로 공개하는 행위
* 콘텐츠의 일부 또는 전부를 이용하여 인쇄물을 만드는 행위
* 콘텐츠의 전부 또는 일부를 녹취 또는 녹화하거나 녹취록을 작성하는 행위
* 콘텐츠의 전부 또는 일부를 스크린 캡쳐하거나 카메라로 촬영하는 행위
* 지인을 포함한 제3자에게 콘텐츠의 일부 또는 전부를 공유하는 행위
* 다른 정보와 결합하여 Upstage Education의 콘텐츠임을 알아볼 수 있는 저작물을 작성, 공개하는 행위
* 제공된 데이터의 일부 혹은 전부를 Upstage Education 프로젝트/실습 수행 이외의 목적으로 사용하는 행위